<a href="https://colab.research.google.com/github/YBen1/Analyse_Flight_Data/blob/main/DayTradeTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I try boild a prediction based on the beginning of the day

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import datetime
from datetime import timedelta

In [ ]:
!pip install yfinance

#ToDo: set end date for today and beginning date at end date - 730 days

In [ ]:
import yfinance as yf

# Set the start and end date
end_date = '2022-03-18'
start_date =  pd.to_datetime(end_date) - timedelta(days=729)

# Set the ticker
ticker = 'AMZN'

# Get the data
data = yf.download(ticker, start_date, end_date, interval = "1h")

# Print 5 rows
data.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
2022-03-17 12:30:00-04:00,3102.815918,3116.090088,3095.425049,3115.000000,3115.000000,347032
2022-03-17 13:30:00-04:00,3116.000000,3143.000000,3106.989990,3125.770020,3125.770020,419986
2022-03-17 14:30:00-04:00,3128.280029,3149.969971,3121.659912,3130.020020,3130.020020,401784
2022-03-17 15:30:00-04:00,3130.850098,3147.546631,3128.385010,3147.389893,3147.389893,440397
2022-03-17 16:00:00-04:00,3144.780029,3144.780029,3144.780029,3144.780029,3144.780029,0


Lets get only the openings

In [ ]:

stock_opening = data.loc[data.index.hour == 9]

In [ ]:
stock_day = data.loc[data.index.hour!= 9]
min_and_max = stock_day.groupby(stock_day.index.date).agg({'High': ['max'], 'Low' : ['min']})

In [ ]:
y = stock_day.groupby(stock_day.index.date).agg({'High': ['max'], 'Low': ['min']})
feature_columns = ['Open', 'High','Low', 'Close' ,'Adj Close', 'Volume']
X = stock_opening[feature_columns]
y.tail()

,High,Low
,max,min
2022-03-11,2972.445068,2907.379883
2022-03-14,2924.000000,2817.679932
2022-03-15,2959.699951,2906.770020
2022-03-16,3063.000000,2947.070068
2022-03-17,3149.969971,3056.445312


In [ ]:
stock_model= DecisionTreeRegressor()
# Fit Model
stock_model.fit(X, y)

DecisionTreeRegressor()

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X,y, random_state = 0)

from sklearn.tree import DecisionTreeRegressor

stock_model = DecisionTreeRegressor(random_state=1)

# Fit iowa_model with the training data.
stock_model.fit(train_X, train_y)

DecisionTreeRegressor(random_state=1)

In [ ]:
val_predictions = stock_model.predict(val_X)

In [ ]:
print(val_predictions[:5])
# print the top few actual prices from validation data
print(val_y.head())
print(val_X.head())

[[3060.85009766 2973.18017578]
 [3202.43994141 3175.77001953]
 [3279.80004883 3212.97509766]
 [3759.9855957  3637.65991211]
 [3256.12011719 3236.17993164]]
                   High          Low
                    max          min
2020-07-28  3059.699951  2995.760010
2020-08-06  3247.469971  3182.010010
2022-02-07  3246.870117  3145.000000
2021-07-26  3708.489990  3683.669922
2021-10-12  3261.080078  3236.280029
                                  Open         High          Low        Close  \
2020-07-28 09:30:00-04:00  3054.270020  3077.092529  3033.625000  3056.310059   
2020-08-06 09:30:00-04:00  3200.500000  3205.000000  3165.429932  3197.399902   
2022-02-07 09:30:00-05:00  3170.399902  3253.814941  3150.000000  3240.989990   
2021-07-26 09:30:00-04:00  3673.169922  3712.080078  3650.000000  3695.385010   
2021-10-12 09:30:00-04:00  3257.000000  3265.449951  3240.000000  3252.620117   

                             Adj Close   Volume  
2020-07-28 09:30:00-04:00  3056.310059   933553 

In [ ]:
from sklearn.metrics import mean_absolute_error
val_mae = mean_absolute_error(val_y, val_predictions)

In [ ]:
val_y['High'].to_numpy()

AttributeError: ignored

In [ ]:
val_mae

27.088164798797123

In [ ]:
def day_trading(closed_at, pred_high, pred_low, real_high, real_low):
  for i in range(len(closed_at)):
    if closed_at[i] - pred_high[i] < pred_low[i] - closed_at[i]  and pred_low[i] < closed_at[i]:
      #print(real_low[i])
      if pred_low[i] > real_low[i]:
        winning_procent = (1 - pred_low[i] /closed_at[i])*100 
        print("you went short at {} and sould at {} with a win of {} procent".format(closed_at[i], pred_low[i], winning_procent))
      else: print('still invested in short')
    else:
      #print(real_low[i])
      if pred_low[i] > real_low[i]:
        winning_procent = ((pred_high[i] /closed_at[i]) -1)*100
        print("you investeted long at {} and sould at {} with a win of {} procent".format(closed_at[i], pred_low[i], winning_procent))
      else: print('still invested in long')


In [ ]:
day_trading(val_X['Close'],val_predictions[:,0],val_predictions[:,1] ,val_y['High'].to_numpy(), val_y['Low'].to_numpy())

still invested in long
still invested in long
you went short at 3240.989990234375 and sould at 3212.97509765625 with a win of 0.8643930608406181 procent
still invested in short
still invested in long
still invested in long
still invested in long
you investeted long at 3592.81005859375 and sould at 3560.0 with a win of 0.5797644084899911 procent
still invested in short
you went short at 3248.409912109375 and sould at 3237.0 with a win of 0.3512460686331931 procent
still invested in long
still invested in short
still invested in short
still invested in long
still invested in short
you investeted long at 3260.070068359375 and sould at 3261.090087890625 with a win of 0.8683805519361654 procent
you investeted long at 2882.639892578125 and sould at 2906.77001953125 with a win of 2.6732461030652743 procent
you went short at 3200.419921875 and sould at 3198.764892578125 with a win of 0.05171287947443037 procent
still invested in long
you went short at 2402.6298828125 and sould at 2391.20996093